## Note dell'autrice
il codice ritorna il dataframe corrispondente alla partita inserita come input. Gli attributi sono:
- la mossa fatta
- il tempo dell'orologio del singolo giocatore al momento della mossa
- il tempo totale trascorso dall'inizio della partita fino al momento in cui la mossa è stata fatta
- evaluation della posizione dal punto di vista del bianco (quindi quando positiva rappresenta una posizione di vantaggio del bianco, se negativa rappresenta una posizione di vantaggio del nero)

L'evaluation delle mosse che non sono scacchi è data in centipawns e indica la differenza tra la mossa migliore data dal motore e quella effettuata, e dividendo per 100 (non me lo sono inventato io, l'ho trovato su internet) è corrispondente quasi perfettamente a quella visibile per esempio su chess.com.
Per questo nel metodo get_evaluation c'è la condizione per capire se la mossa sia mate o no.

L'unica incertezza riguarda gli scacchi (visibili nel dataframe perchè hanno davanti #), dove non riesco a interpretare l'evaluation, che dovrebbe essere molto più alta e soprattutto non essere decrescente nelle ultime mosse della partita che hanno portato alla vittoria del bianco (nella partita che gli ho passato come input per il test finale "valesepicacchi_vs_dmac5555_2022.05.07 (1).pgn").
Se capite perchè e come dare un punteggio che sia coerente con l'imminente vittoria ditemelo e aggiungetelo.

Altro problema è la libreria di stockfish, che se importavo normalmente e mettevo la directory di dove si trovava mi dava un errore nel testo molto strano, quindi sono stata costretta ad aggiungerla nel folder e così per motivi misteriosi funziona. In questa riga di codice (stockfish = Stockfish()) della seconda cella se non viene passato un parametro va automaticamente a cercare nel folder un file executable che si chiami stockfish e quindi non mi da problemi. Anche questa riga (engine = chess.engine.SimpleEngine.popen_uci("stockfish")) sempre della seconda cella funziona solo in questo modo. Ovviamente il file stockfish.exe è la libreria per windows quindi non credo funzionerà per un mac. Se a voi funziona importando la libreria normalmente potete inserire la directory della libreria nei due metodi citati sopra, passandola come parametro all'interno delle parentesi.
Nei prossimi giorni proverò a risolvere questo problema ma ci ho passato tre ore e mi ero rotta :)

In [1]:
import chess
import chess.pgn
import pandas as pd
import numpy as np
import chess.engine
from stockfish import Stockfish


In [2]:
#this method takes as input the board and the last move so to update the board and compute the evaluation
def get_evaluation(board,move):

    stockfish = Stockfish()     #if no parameter passed, takes as input the executable file called stockfish from the same directory, if found
    board.push_san(move)        #insert the move in the board
    stockfish.set_fen_position(board.fen())  #bpard.fen() applies a PGN to FEN conversion
    engine = chess.engine.SimpleEngine.popen_uci("stockfish")
    info = engine.analyse(board, chess.engine.Limit(1))     #gets the infos of the game
    
    #if the position on the board is not a mate the score is in centipawns, then it's divided by 100 to get the actual score
    #Gets the score from White’s point of view
    if chess.engine.PovScore.is_mate(info['score']):
        eval = chess.engine.PovScore.white(info['score'])
    else:
        eval = int(str(chess.engine.PovScore.white(info['score'])))/100

    return board, eval      #returns the board updated and the evaluation of the position

In [3]:
#create_dataFrame takes as input the name of the file pgn containing the game and returns a dataframe
def create_dataFrame(game_name): 

    #opens and reads the pgn file
    pgn = open(game_name) 
    game = chess.pgn.read_game(pgn)
    
                                   
    array = []      #initialize the array that will contain the data
    index_values = [] 
    i=0
    time_white = 0      #to keep track of the time for each player
    time_black = 0      #to keep track of the time for each player
    board = chess.Board()       #initialize the chessboard

    #iterate through the moves
    for node in game.mainline():        
        
        move = str(node.move)       #represents the last move
        board,evaluation = get_evaluation(board,move)       #the get_evaluation method returns the updated board and the evaluation of the current position

        if i%2 == 0:        #if the white is moving we update his time
            index_values.append("white")
            time_white = 600-node.clock()

        else:           #if the black is moving we update his time
            index_values.append("black") 
            time_black = 600-node.clock()

        #evaluation = get_evaluation(board,move)
        #we append an element in the array (a row in the dataframe) with move, time the player used, total time since the game started and evaluation of the position
        array.append([str(node.move),round(600-node.clock(),4),round(time_white+time_black,4),evaluation])   
        i+=1

    column_values = ['move','time','real time','evaluation'] #create the columns names
    array = np.array(array)      #convert our array to a numpy array
    #create a dataframe
    df = pd.DataFrame(data = array, index = index_values, columns = column_values) 
    return df,board


df,board = create_dataFrame("valesepicacchi_vs_dmac5555_2022.05.07 (1).pgn")   
df



,move,time,real time,evaluation
white,e2e4,0.0,0.0,0.16
black,e7e5,2.7,2.7,0.92
white,g1f3,1.7,4.4,0.51
black,d7d6,6.7,8.4,0.8
white,b1c3,4.2,10.9,0.01
black,c8g4,7.8,12.0,0.94
white,f1e2,6.6,14.4,0.24
black,g4f3,9.3,15.9,0.96
white,e2f3,8.1,17.4,0.68
black,a7a5,11.2,19.3,1.59
